## 近似ベイズ推論  
解析的な推論ができないニューラルネットワークのような複雑な確率モデルでの近似的計算手法
### 4.1 サンプリングに基づく推論手法
観測データを${\bf X}$，パラメータや潜在変数の集合を${\bf Z}$とした時，  
$p({\bf Z}|{\bf X})$を求めたい  
→解析的に求められないので$p({\bf Z}|{\bf X})$から複数のサンプルを得ることで分布の特性を調べる  

#### 4.1.1単純モンテカルロ法
分布p(z)に関して
$f(z)$
の期待値$\int f(z)p(z)dz$を求める
1. $\int f(z)p(z)dz$の解析的な積分計算は難しい
2. 分布$p(z)$からのサンプリングは容易  

という状況 
\begin{eqnarray}
  \int f(z)p(z)dz \approx\ \frac{1}{T} \sum_{t=1}^T f(z^{(t)}) \tag{1}\\
\end{eqnarray}

- p(z)から十分大きなT個のサンプルを抽出して近似している  

$p(X,\theta) = p(X|\theta)p(\theta)$の周辺尤度$p(X)$の評価に実際使おうとすると  
- $p(\theta)$は幅広く撮る必要がある  
- $p(X|\theta)$は特定の狭い$\theta$でしか大きな値を取らない  
ので実用されることは少ない

#### 4.1.2 棄却サンプリング  
目標分布
$$
p(z) = \frac{1}{Z_p} \tilde{p}(z) \tag{2}
$$
からのサンプルを得たい

- p(z)を直接計算できないので正規化されていない関数$\tilde{p}(z)$を利用

##### 手順
$k>0$を$kq(z)>\tilde{ p} (z)$となるように設定  
サンプリングが簡単に行える仮の分布q(z)を設定  
1. $z^{(t)} ~ q(z)$を得る  
2. 一様分布からのサンプル$\tilde{u} ~ Uni(u|0, kq(z^{(t)}))$を得る  
3. $\tilde{u}\leq\tilde{p}(z^{(t)})$であれば受容される時，サンプルが受容される確率は
$$
\int q(z)\frac{\tilde{q}(z)}{kq(z)}dz = \frac{1}{k}\int \tilde{p}(z)dz \tag{3}
$$

となる  
棄却サンプリングはニューラルネットワーク等の複雑なモデルで高次元の変数のサンプリングが必要とされる時サンプルの受容率が非常に低くなる

#### 4.1.3 自己正規化重点サンプリング  
p(z)から直接サンプリングが得られなくても使える手法  
式(1)の期待値そのものを効率的に計算することを目標とする  
式(1)は
\begin{eqnarray}
    \int f(z)p(z)dz = \int f(z) \frac{p(z)}{q(z)} q(z)dz\\
    = \mathbb{E} _{q(z)} \left[ f(z) \frac{\tilde{p}(z)}{q(z)}\right] \\
    = \frac{Z_q}{Z_p} \mathbb{E}_{q_(z)} \left[ f(z) \frac{\tilde{p}(z)}{\tilde{q}(z)} \right] \\
    \approx \frac{Z_q}{Z_p} \frac{1}{T} \sum_{t=1}^T f(z^{(t)})w^{(t)} \tag{4}
\end{eqnarray}

となる
- $w^{(t)} = \frac{\tilde{p}(z^{(t)})}{\tilde{q}(z^{(t)})}$とおいた  

正規化項の比$\frac{Z_p}{Z_q}$は
\begin{eqnarray}
    \frac{Z_p}{Z_q} = \int \frac{\tilde{p}(z)}{Z_q}dz\\
    =\int \frac{\tilde{p}(z)}{tilde{q}(z)} q(z)dz\\
    =\mathbb{E}_{q(z)} \left[ \frac{\tilde{p}(z)}{tilde{q}(z)} \right]\\
    \approx \frac{1}{T} \sum_{t=1}^T w^{(t)} \tag{5}
\end{eqnarray}

のようにq(z)からT個のサンプルを使って近似的に求められる  
- 棄却サンプリングは直観的かつ実装も容易だが一次元程度の簡単な積分近似にしか適用できない

#### 4.1.4 マルコフ連鎖モンテカルロ法
- 高次元の空間で効率的にサンプリングを行うための手段  

式(6)が成り立つ時，分布$p_*(z)$は定常分布であるという

\begin{eqnarray}
    p_*(z) =\int T(z', z)p_*(z')dz'. \tag{6}
\end{eqnarray}

- p(z)に一次マルコフ連鎖を仮定している
- $ T(z', z)$は遷移確率

遷移確率Tによって任意の初期状態$p_0$から定常分布$p_*$に収束できなければならない  
->エルゴード性  
   - 有限回数で遷移できること
   - 全ての状態が固定の周期性を持たないこと
   - 同じ状態に有限回で戻ることができること

#### 4.1.5 メトロポリス・ヘイスティングス法  
$p(x) \propto \tilde{p}(x)$で$\tilde{p}(x)$は計算可能  
遷移確率T(z',z)が求められない時には遷移の提案分布a(z|z')を使うことができる  

##### アルゴリズム
1. 提案分布$q(･|z^{(t)})$から次のサンプル候補$z_*$をサンプリングする
2. 比率rを計算する  
$$
r = \frac{\tilde{p}(z_*)q(z^{(t)}|z_*)}{\tilde{p}(z^{(t)})q(z_*|z^{(t)})} \tag{7}
$$
3. 提案された点$z_*$を確率min(1,r)によって  
$z^{(t+1)}\leftarrow z_*$として受容し，そうでない場合は$z^{(t+1)}\leftarrow z^{(t)}$とする

提案分布としてはガウス分布が使われることが多い  

#### 4.1.6 ハルミトニアンモンテカルロ法
解析力学的な物体の軌道のシミュレーションとメトロポリス・ヘイスティングス法を組み合わせたサンプリング手法  
事後分布の微分情報を利用することでガウス分布を適用したメトロポリス・ヘイスティングス法より効率的に事後分布の空間を探索できる  
##### ハルミトニアン  
ハルミトニアンHは運動エネルギーTとポテンシャルエネルギーVとして，全エネルギーを  
$$H(z,p,t)=U(z)+K(p)\tag{8}$$
のように位置ベクトルz，運動量ベクトルp，時間tによって表した関数のこと  
U(z)は位置によって決まるポテンシャルエネルギー，K(p)は運動エネルギー  
質量1のとき$K(p)=\frac{1}{2}p^tp$

ハルミトニアンの偏微分
\begin{eqnarray}
    \frac{dp_i}{dt} = -\frac{dH}{dz_i},\\
    \frac{dz_i}{dt} = \frac{dH}{dp_i}\tag{9}
\end{eqnarray}

(9)を(8)に代入すれば
\begin{eqnarray}
\frac{dp_i}{dt} = -\frac{dU}{dz_i}, \\
\frac{d_zi}{dt} = \frac{dK}{dp_i}\tag{10}
\end{eqnarray}

(9)が解析的に得られないとして，数値シミュレーションによって物体の軌道を計算する．  
##### オイラー法
\begin{eqnarray}
p_i(t+\epsilon) = p_i(t) + \frac{dp_i}{d\epsilon} \tag{A1}
\end{eqnarray}

(A1)は時間Tを$t+\epsilon$として表し(離散化)解を定義する

###### ピカールの逐次近似法

- オイラー法の理解にあたって必要
- 任意の区間内で初期条件$x(\tau) = \xi$を満たす解x(t)を求める
    - これは$x(t) = \xi + \int_\tau^t f(s, x(s)) ds$を求めることと同値

#### オイラー法を用いて物体の軌道を計算
\begin{eqnarray}
p_i(t+\epsilon) = p_i(t) + \left.\frac{dp_i}{d\epsilon}\right|_t = p_i(t) - \epsilon \left.\frac{dU}{dz_i}\right|_{z_i(t)} \\
z(t + \epsilon) = z_i(t) + \epsilon \left.\frac{dp_i}{dt}\right|_t = z_i(t) + \epsilon p_i(t) 
\end{eqnarray}

として時刻$\epsilon>0$の挙動を近似的に予測する  
→離散化による数値誤差が大きい  
  <span style="color: red; ">"近似"は離散化のこと？</span>
#### リープフロッグ法
オイラー法の改良版(オイラー法は一次精度，リープフロッグ法は2次精度)  
時間可逆性
- n回積分したのち時間を逆にしてn回積分すれば元の位置に戻る  

シンプレクティック性
- エネルギー保存的なやつ 時間とともに誤差が増大することがない  

という性質を持つのでハミルトニアンモンテカルロ法に利用される
→比率rの計算時に確率変数の変換に伴うヤコビ行列お決定式を計算する必要がなくなるので計算効率化につながる  
\begin{eqnarray}
p_i \left( t+\frac{\epsilon}{2} \right) = p_i(t) -\left.\frac{\epsilon}{2}\frac{dU}{dz_i} \right|_{zi(t)}\tag{A2}\\ 
z(t + \epsilon) = z_i(t) + \epsilon p_i\left(t + \frac{\epsilon}{2}\right)\tag{A3} \\
p_i(t+\epsilon) = p_i\left(t+\frac{\epsilon}{2}\right) -\left.\frac{\epsilon}{2}\frac{dU}{dz_i}\right|_{z_i(t+\epsilon)}\tag{A4}
\end{eqnarray}

この手続きをL回繰り返すことで時刻$\epsilon L$先の物体の位置$z_*$と運動量$p_*$を計算できる

#### 4.1.6.2 サンプリングアルゴリズムへの適用
リープフロッグ法を使ったシミュレーションをサンプリングアルゴリズムに適用する  
サンプルを得たい確率分布$p(z)\propto \tilde{p}(z)$に対して補助分布${\bf p}$を導入し  
$p(z,p) = p(z)p({\bf p})$のように拡張する  


p(z)とp(p)は独立なので同時分布p(z)p(p)から得られるzのサンプルは周辺分布p(z)から得られたものと同一視できる  
$p({\bf p}) = N({\bf p}|{\bf 0, I})$とし，さらに$\log \tilde{p}(z)=-U(z)$とおいて同時分布を計算すると  

\begin{eqnarray}
    p(z,p) = exp(\log p(z) + \log p({\bf p})) \\
    \propto exp \left( \log \tilde{p}(z) - \frac{1}{2}{\bf p}^t{\bf I}{\bf p}\right) \\
    = exp\left( -U(z) - K(p) \right)\\
    = exp(-H(z,p)) \tag{11}
\end{eqnarray}

となり，式(11)は式(8)のハミルトニアンを表す  

運動量${\bf p}$をガウス分布に従ってサンプリングした後，ハルミトニアンのシミュレーションを行えば新しいサンプル点の候補$(z_*,p_*)$を得ることができる  


この時シミュレーション上では物体はハミルトニアンHをほぼ一定に保ったまま軌道を描くので式(7)の比率r  
\begin{eqnarray}
    r = \frac{p(z_*,p_*)}{p(z,p)}\\
    = exp(-H(z_*,p_*) + H(z,p))
\end{eqnarray}
は常に1に近い値をとる  
→受容率が非常に高い


$\epsilon$が小さいほどシミュレーションとの誤差が小さくなり受容率が上がるが遷移の移動量が小さくなるため効率的な探索が行えなくなる  
Lが大きいほど移動量が大きくなるが計算コストが大きくなる  

ハミルトニアンモンテカルロ法は事後分布の微分さえ計算できれば適用できる  
→汎用性が高い  
離散線代変数などの微分できない変数をそのままで扱うことはできない  

#### 4.1.6.3 ランジュバン動力学法
L=1とした場合をランジュバンモンテカルロ法もしくはランジュバン動力学法という  
式A2をA3に代入すると，

\begin{eqnarray}
    z_{*i} = z_i(t+\epsilon) \\
    = z_i(t) + \epsilon \left\{ p_i(t) - \left.\frac{\epsilon}{2}\frac{dU}{dz_i}\right|_{z_i(t)} \right\} \\
    = z_i(t) - \frac{\epsilon ^2}{2}\left.\frac{dU}{dz_i}\right|_{z_i(t)} + \epsilon p_i(t)
\end{eqnarray}

#### 4.1.7 ギブスサンプリング
確率分布p(z)から直接Z全体をサンプリングすることが難しい場合に  
ZをM個の部分集合に分けて逐次的にサンプリングを行う手法のこと  

- サンプルを得たい変数の数が膨大な時
- 複数の確率分布が組合わさった巨大な確率モデルからサンプルを得たい場合
に特に有効  

### 4.2 最適化に基づく推論手法  
マルコフ連鎖モンテカルロ法は無限に計算を続ければサンプルを真の分布から得られたものと同一視して良い  
→適切なサンプルサイズが決めにくい，計算コストが大きいなどの不便  
⇔勾配情報を用いた数値最適化は実験的に速い収束性能を持つ

#### 4.2.1 変分推論法  
事後分布に出現する解析不能な積分を最適化問題に置き換えることで近似的に計算する  
周辺尤度$p({\bf X})$の計算には潜在変数Zの積分除去$p({\bf X}) = \int p({\bf X, Z})d{\bf Z}$が必要となるがモデルが複雑になるとこの積分は解析的に実行できないのでELBO(Evidence lower bound)という対数周辺尤度を考える

##### ELBO
変分推論のタスクを周辺尤度の下限の最大化とする  
\begin{eqnarray}
    確率モデルp({\bf X},{\bf Z}) \\{\bf X}:観測データ \\ {\bf Z}:未観測の変数  \\
    {\bf Z}に関する確率分布q({\bf Z})を仮定し，このモデルの周辺尤度p({\bf X})を求めると\\  
    \log p({\bf X}) = \log \int p({\bf X}, {\bf Z})d{\bf Z} \\
    = \log \int q({\bf Z}) \frac{p({\bf X},{\bf Z})}{q({\bf Z})} d{\bf Z} \\
    \geq \int q({\bf Z}) \log \frac{p({\bf X},{\bf Z})}{q({\bf Z})} d{\bf Z} \tag{イェンセンの不等式を用いた}\\
    = L[q({\bf Z})] \tag{12}
\end{eqnarray}
この式における$L[q(Z)]$を任意の確率分布$q(Z)$に対するELBOと呼ぶ  
複雑な確率モデルでは周辺尤度を厳密に計算できないこともあるので代わりにL[q(Z)]をなるべく大きくするようなq(Z)を求めることで対数周辺尤度$\log p(X)$を近似的に計算することができる  

対数周辺尤度とELBOの差は
$$
\log p(X) - L[q(Z)] = KL[q(Z)||p(Z|X)]
$$
なのでq(Z)とp(Z|X)のKLダイバージェンスになっている

式(12)の近似分布qの置き方は色々ある  
潜在変数の集合ZをM個に分割する近似がよく使われる(らしい)  
- 事後分布に独立性を仮定している  
- 平均場近似と呼ばれる
- 分割された集合間の相関は捕らえられない反面計算に必要なメモリは少なく済む  
    - 近似分布の表現力が限定されているので真の事後分布が複雑な場合近似精度に限界がある  
$$
q(Z) = \sum_{i=1}^M q(Z_i) \tag{13}
$$
平均場近似では各近似分布($q(Z_1),...,q(Z_M)$)を交互に更新していく  
- ギブスサンプリングに似た手続き

#### 4.2.2 平均場近似による潜在変数モデルの学習 次元削減  



観測データ$X=\{x_1, x_2,...,x_N\}$が入力変数$Z=\{z_1, z_2,...,z_N\}$の線形結合および固定のノイズ$\sigma_x^2$によって決まると仮定する  
\begin{eqnarray}
    p(X|Z,W) = \sum_{n=1}^N p(x_n|z_n, w_n) \\
    =\sum_{n=1}^N N(x_n|W_{z_n}, \sigma_x^2 I) \tag{14}
\end{eqnarray}

入力変数Zは未観測の潜在変数であるとし，  
潜在変数，パラメータはそれぞれガウス分布によって生成されているとする  
$$
p(Z) = \prod_{n=1}^N N(z_n | O,I)\\
p(W) = \prod_{i,j} N(w_{i,j}|o,\sigma^2_w) \tag{15}
$$

真の事後分布を
$$
p(Z,W|X) \approx q(Z)q(W) \tag{16}
$$
のようにqで分解近似

この時，p(X)をZ,Wで周辺化した時の周辺尤度p(X)の下界Lは  
\begin{eqnarray}
    L = \int q(Z)q(W) \log \frac{p(X,Z,W)}{q(Z)q(W)} dZdW\\
    = \mathbb{E}_{q(Z)q(W)} [\log p(X|Z,W)] - KL[q(Z)||p(Z)] -KL[q(W)||p(W)] \tag{17}
\end{eqnarray}

となる  

変分推論法では$q(Z_i)$と$q(W_i)$を交互に固定して値を更新していく  

##### 例：$q_i(Z)$を固定して$q_{i+1}(W)$の値を更新する  
\begin{eqnarray}
    L_{q_i(Z)} = \mathbb{E}_{q_i(Z)q_{i+1}(W)} [\log p(X|Z,W)] - KL[q_{i+1}(W)||p(W)] +c \\
    = - 
    \mathbb{E}_{q_i(W)} \left[ \frac{ \log exp(\mathbb{E}_{q_i(Z)} [\log p(X|W,Z)])p(W)}{q_{i+1}(W)} \right] +c \\
    = - KL[q_{i+1}(W)||r_i(W)] + c \tag{18}\\
    ただしr_i(w) \propto \log exp(\mathbb{E}_{q_i(Z)} [\log p(X|W,Z)])p(W)\\
    また，\int r_i(W)d(W) = 1
\end{eqnarray}
ここの式の上から２行目参考書の方は誤植だと思われ(結論は合ってる)

よって式(18)の最大化はKLダイバージェンスの最小化と等価なので最適解は
$$
q_{i+1}(W) = r_i(W) \tag{19}
$$

式(19)のパラメータの近似事後分布の更新は変分Mステップと呼ぶ  
同様にして$q_{i+1}(W)$を固定して$q_{i+1}(Z)$を求めるのをEステップと呼ぶ  

線形次元削減モデルではEステップ，Mステップでの$r_i(W),r_i(Z)$が解析的に計算でき，ガウス分布になることがわかる  

#### 4.2.2.2 混合ガウス分布への応用  
連続な潜在変数Zの代わりに離散の潜在変数Sを用いたクラスタリングの場合  
各潜在変数を$s_n \in {0,1}^K$かつ$\sum_{k=1}^K=1$として混合ガウス分布を考える  
\begin{eqnarray}
    p(X|S,W) = \prod_{n=1}^N p(x_n|s_n,W) \\
    = \prod_{n=1}^N N(x_n|W_{s_n}, \sigma^2_x I) \tag{20}\\
    各潜在変数はカテゴリ分布
    p(S) = \prod_{n=1}^N Cat(s_n|\pi)\\
    各クラスタk=1,2,...,Kのパラメータはp(W)=\prod_{i,j} N(w_{i,j}|o,\sigma_w^2)\\に従うとする
\end{eqnarray}

このモデルの事後分布も平均場近似を用いて$p(S,W|X) \approx q(S)q(W)$のように分解すれば線形次元削減と同じ方法で変分EMアルゴリズムが得られる  

#### 4.2.4 ラプラス近似  


事後分布をガウス分布で近似する手法  
\begin{eqnarray}
    p(Z|X) \approx N(Z|Z_{MAP}, \{\Lambda(Z_{MAP})\}^{-1} \tag{21} \\
    ただし，\Lambda(Z) = -\nabla^2_Z \log p(Z|X) 
\end{eqnarray}

$¥labmda(Z)$は対数事後分布のヘッセ行列を負にしたもの  
発想としてはp(Z|X)が最大値をとる値の付近に限定すればp(Z|X)をガウス分布に近似できるよねというもの  
もっとちゃんと言うとこれは$Z_{MAP}$周りのテイラー展開で2次近似することに対応している  
\begin{eqnarray}
    \log p(Z|X) \approx \log p(Z_{MAP} |X) +(Z-Z_{MAP})^T\nabla_Z \log p(Z|X)|_{Z=Z_{MAP}} + (Z-Z_{MAP})^T\nabla^2_Z \log p(Z|X)|_{Z=Z_{MAP}}(Z-Z_{MAP}) \\
    = \log p(Z_{MAP}|X)+(Z-Z_{MAP})^T\nabla_Z^2 \log p(Z|X)|_{Z=Z_{MAP}}(Z-Z_{MAP}) \tag{22}
\end{eqnarray}
($Z_{MAP}で\nabla_Z \log p(Z|X) = 0$を利用)

式(22)の指数をとると
$$
p(Z|X) \propto exp(-(Z-Z_{MAP})^T \Lambda (Z_{MAP})(Z-Z_{MAP}))
$$

と近似され，式(21)のガウス分布による表現が得られる  
- ラプラス近似は既存の正規化やMAP推定に基づくパラメータの学習手法から簡単に拡張を行える
- ヘッセ行列を求めるコストが大きい(パラメータ数の二乗のオーダー)
- 離散や非負の実数上で定義される確率分布に直接は適用できない(ガウス分布による近似なので)

#### 4.2.4 モーメントマッチングによる近似
求めたい複雑な確率を次のような指数分布による近似分布q(z)で表す
$$
q(z;\eta) = h(z)exp(\eta ^T t(z) -a(\eta)) \tag{23}
$$

KLダイバージェンスを$KL[p(z)||q(z;\eta)] \tag{24}$とおく  
- 普段のKLダイバージェンスとp,qの順番が逆になっている  

式(24)に式(23)を代入して自然パラメータ$\eta$に関してKLダイバージェンスを最小化したい  
$\eta$について整理すると
$$
式(24) = -\mathbb{E}_p[\log q(z;\eta)] + \mathbb{E}_p [\log p(z)] \\
= -\eta ^T \mathbb{E}_p [t(z)] + a(\eta) + c
$$

$\eta$に対する勾配を計算し，ゼロと置けば, 対数分配関数$a(\eta)$の$\eta$に関する勾配が十分統計量t(z)の期待値であることから  
$$
\mathbb{E}_q[t(z)] = \mathbb{E}_p[t(z)]
$$

指数型分布族を用いて分布p(z)を式(24)の基準で最適に近似するためには，$\mathbb{E}_p[t(z)]$(p(z)のモーメントを計算し，その結果を用いて指数型分布族のパラメータ$\eta$を決定すれば良い  

#### 4.2.4.2 仮定密度フィルタリング

モーメントマッチングによる近似推論の最もシンプルな近似逐次手法  
データ集合$D_1$を観測した後の事後分布は  
$$
p(\theta|D_1) \propto p(D_1 | \theta ) p(\theta) \tag{25}
$$

尤度関数$p(D_1|\theta)$と事前分布$p(\theta)$の間に共役性がが成り立てば解析的に事後分布を更新していける  
→共役性の成り立たないモデルのことを考えていく  
$p(\theta|D_1)$に対する近似分布$q_1(\theta)$を設定する  
$$
q_1(\theta) \approx r_1(\theta) = \frac{1}{Z_1} p(D_1|\theta)p(\theta) \tag{26} \\
ただしZ_1 =\int p(D_1|\theta )p(\theta)d\theta
$$

式(26)の右辺を計算して，それと一致するモーメントを持つように左辺$q_1$のパラメータを設定する  

続く$q_i$も以下のように計算していく
$$
q_{i+1}(\theta) \approx r_{i+1}(\theta) = \frac{1}{Z_{i+1}} p(D_{i+1}|\theta)q_i(\theta) \tag{27} \\
$$
このように計算していけば同じ分布の形式を保ったまま近似事後分布の更新ができる  

#### 4.2.4.3 ガウス分布の例  
式(27)で分布$q(\theta)$がガウス分布の場合  
$$q_i(\theta) = N(\theta|\mu_i,v_i)\tag{28}$$
正規化定数$Z_{i+1}$は
\begin{eqnarray}
Z_{i+1} = \int f_{i+1}(\theta)q_i(\theta)d\theta \\
=\int f_{i+1}(\theta)\frac{1}{\sqrt{2\pi v_i}}exp\left(-\frac{(\theta-\mu)^2}{2v_i}\right) d\theta \tag{28}
\end{eqnarray}

$Z_i$の対数をパラメータ$\mu_i$で偏微分すれば  
\begin{eqnarray}
\frac{\partial}{\partial\mu_i}\log Z_{i+1} = \frac{1}{Z_{i+1}} \int f_{i+1}(\theta)N(\theta|\mu_i,v_i)\frac{\theta - \mu_i}{v_i}d\theta \\
= \frac{\mathbb{E}_{r_i+1}[\theta]-\mu_i}{v_i}\tag{29}
\end{eqnarray}

$Z_i$の対数をパラメータ$v_i$で偏微分すれば  
\begin{eqnarray}
\frac{\partial}{\partial v_i}\log Z_{i+1} = \frac{1}{Z_{i+1}} \int f_{i+1}(\theta)N(\theta|\mu_i,v_i)\left\{- \frac{1}{2v_i} + \frac{(\theta - \mu_i)^2}{2v_i^2}\right\}d\theta \\
= -\frac{1}{2v_i} + \frac{1}{2v_i^2}\{\mathbb{E}_{r_i+1}[\theta^2]-2\mu_i\mathbb{E}_{r_i+1}[\theta]+ \mu_i^2 \} \tag{30}
\end{eqnarray}

よって式(29)，(30)より，一次のモーメントは
$$
\mathbb{E}_{r_i+1}[\theta] = \mu_i + v_i \frac{\partial}{\partial \mu_i} \log Z_{i+1} \tag{31}
$$

二次のモーメントは  
$$
\mathbb{E}_{r_i+1}[\theta^2] = 2v_i^2 \frac{\partial}{\partial v_i} \log Z_{i+1} + v + 2\mu_i \mathbb{E}_{r_i+1}[\theta] -\mu^2 \tag{32}
$$
となる  

式(29)と式(31)の結果を用いればモーメントマッチングによって得られる新しい分布$q_{i+1}$のパラメータは  
$$
\mu_i = \mathbb{E}_{r_i+1}[\theta]\\
v_{i+1} = \mathbb{E}_{r_i+1}[\theta^2] -\mathbb{E}_{r_i+1}[\theta]^2
$$
なので式(31),(32)を用いて更新できる  
ただし，これらの値が計算には正規化定数$Z_{i+1}$の計算ができることが条件となる

#### 4.2.5 プロビット回帰  
プロビット回帰の尤度関数は  
$$
p(Y|X,w) = \prod_{n=1}^N p(y_n|x_n,w) \\
= \prod_{n=1}^N \Phi (y_n w x_n)\tag{33}\\
\Phi (x) = \int_{-\infty}^x N(t|0,1)dt\\
パラメータの事前分布はガウス分布 p(w) = N(w|0,v_0)
$$

このモデルの周辺尤度$Z = \int p(Y|X,w)p(w)dw$が解析的に計算できない  
代わりにモーメントマッチングによる近似的な事後分布の更新で尤度の項$f_i(w)=p(y_i|x_i,w)$を1つずつ追加していくことを考える  
パラメータの事後分布の近似にはガウス分布を使用する  
\begin{eqnarray}
    q_1(w) \approx p(w|y_1,x_1) = \frac{1}{Z_1}p(y_1|x_1,w)p(w)\\
    q_{i+1}(w) \approx \frac{1}{Z_{i+1}}f_{i+1}(w)q_i(w) \tag{34}
\end{eqnarray}

ガウス分布を用いた近似更新を行うためには式(31),(32)より$Z_i$の微分が必要となる  
プロビット回帰において正規化定数$Z_i$は解析的に計算できるので実行可能  

#### 4.2.6 期待値伝播法  
仮定密度フィルタリングをパッチ学習できるように一般化した方法
- 学習データの順序に依存しない  
- 学習に使ったデータを捨てないので仮定密度フィルタリングより精度の高い近似事後分布を得られる

モデル
$$
p(X,\theta) = p(\theta)\prod_{n=1}^N p(X_n|\theta) = \prod_{n=1}^N f_n(\theta) \tag{35}
$$

$f_n$は因子(factor)と呼ばれるもので以下の対応関係を持つ  
$$
f_n(\theta) = \begin{cases}p(\theta),\quad if \quad　n=0\\
p(x_n|\theta),\quad if\quad n>0
\end{cases}
$$

因子を使って書くとこのモデルの事後分布は  
$$
p(\theta|X) = \frac{p(\theta)\prod_{n=0}^Nf_b(\theta)}{p(X)} \tag{36}
$$

となる  
この事後分布に対する近似分布を近似因子$\tilde{f}_n$の積で表現する  
$$
q(\theta) = \frac{1}{Z} \prod_{n=1}^N\tilde{f}_n(\theta) \tag{37}
$$
Zは正規化を保証する定数  
ガウス分布を近似因子として選ぶと
$$
\tilde{f}_n = N(\theta| \mu_n , \Sigma_n) \tag{38}
$$
式(37)より，N＋1個の近似因子の積を正規化することによって得られる$q(\theta)$もガウス分布になる

仮定密度フィルタリングのように逐次的に更新していく場合  
更新途中における現在の近似分布からi番目の現在の近似因子を取り除く
$$
q_{\\i}(\theta) = \prod_{j\neq i}\tilde{f}_j(\theta) = \frac{q_{old}(\theta)}{\tilde{f}_i(\theta)}
$$

近似因子を取り除いた$q(\theta)$に対してモデルの因子$f_i(\theta)$を追加し，\\正規化したものを$r(\theta)=\frac{1}{Z_i}f_i(\theta)_{\\i}(\theta)$とおく  
$r(\theta)$のモーメントを計算してこれを新しい近似分布のモーメントにし，次の近似因子を
$$
\tilde{f}_i(\theta) \leftarrow Z_i \frac{q_{new}(\theta)}{q_{\\i}(\theta)}
$$
として更新する